In [1]:
# Necessary libraries for data manipulation, plotting, molecular dynamics analysis, and order parameter constants.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import opc
import MDAnalysis as mda

class OrderParameters:
    def __init__(self, u, atomlists, resname, selection):
        """Constructor for the OrderParameters class.
        
        Parameters:
        - u: MDAnalysis universe object containing the trajectory and topology information.
        - atomlists: List of atoms for which the order parameter is to be computed.
        - resname: Name of the residue (lipid type).
        - selection: Selection string to filter specific molecules from the universe.
        """
        
        self.u = u
        self.atomlists = atomlists
        self.resname = resname
        self.selection = selection
        
        # Process the provided atom lists to extract relevant atom information.
        self.C_numbers, self.Cs, self.Hs_f, self.repeat = self.process_atom_lists()

    def process_atom_lists(self):
        """Processes the atom list to determine carbon numbers, carbon atom names, hydrogen atom names, and repetition counts."""
        
        C_numbers = []  # List to store carbon numbers
        Cs = []  # List to store carbon atom names
        Hs = []  # List to store hydrogen atom names associated with each carbon
        repeat = []  # List to store how many times each hydrogen atom appears
        
        for atoms in self.atomlists:
            C_number = atoms[0][2:]  # Extract carbon number from atom name
            C_numbers.append(int(C_number))
            Cs.append(atoms[0])  # Store carbon atom name
            Hs.append(atoms[1:])  # Store hydrogen atom names
            repeat.append(len(atoms) - 1)  # Count of hydrogen atoms per center carbon atom
        
        # Flatten the list of hydrogen atoms
        Hs_f = [item for sublist in Hs for item in sublist]
        
        # Check if the total repetitions match the number of hydrogen atoms
        assert int(np.sum(repeat)) == len(Hs_f), "Mismatch in repeats"
        return C_numbers, Cs, Hs_f, repeat

    def compute_OP(self):
        """Compute order parameter for each frame in the trajectory for selected molecules."""
        
        # Select the molecules that match the selection criteria.
        all_molecules = self.u.select_atoms(self.selection, updating=True)
        output = []

        for ts in self.u.trajectory:  # Iterate through each frame of the trajectory
            valid_indices_group1 = []  # List to store indices of carbon atoms
            valid_indices_group2 = []  # List to store indices of hydrogen atoms
            
            for molecule in all_molecules.residues:
                atoms_in_molecule = molecule.atoms
                # Check if the molecule contains all atoms of interest
                if all(atom.index in all_molecules.indices for atom in atoms_in_molecule):
                    valid_indices_group1.extend(molecule.atoms.select_atoms("name " + " ".join(self.Cs)).indices)
                    valid_indices_group2.extend(molecule.atoms.select_atoms("name " + " ".join(self.Hs_f)).indices)
            
            # Create atom groups based on the valid indices
            group1 = self.u.atoms[valid_indices_group1]
            group2 = self.u.atoms[valid_indices_group2]

            natoms = len(self.Cs)  # Count of center atoms
            nmols = int(len(group1.positions) / natoms)  # Count of molecules
            repeats = self.repeat * nmols  # Calculate repetitions for this frame

            # Calculate cosine of the angle between molecular segment and bilayer normal (z-axis)
            p1 = np.repeat(group1.positions, repeats, axis=0)  # Repeat carbon atom positions
            p2 = group2.positions  # Hydrogen atom positions
            dp = p2 - p1  # Difference in positions
            norm = np.sqrt(np.sum(np.power(dp, 2), axis=-1))
            cos_theta = dp[..., 2] / norm

            # Compute order parameter using the standard formula
            S = -0.5 * (3 * np.square(cos_theta) - 1)

            # Average the order parameter over all hydrogen atoms associated with each carbon
            new_S = self._average_over_hydrogens(S, repeats)
            new_S.shape = (nmols, natoms)
            results = np.average(new_S, axis=0)
            output.append(results)

        # Calculate average order parameter over the trajectory
        avg = np.average(output, axis=0)
        return np.transpose([self.C_numbers, avg])

    def _average_over_hydrogens(self, x, reps):
        """Average a property (e.g., order parameter) over hydrogen atoms.
        
        Parameters:
        - x: Array containing the property for each hydrogen atom.
        - reps: List indicating how many times each hydrogen atom appears.
        """
        
        assert len(x) == int(np.sum(reps)), 'Mismatch in repeats'
        i = 0
        out = []
        for rep in reps:
            tmp = []
            for r in range(rep):
                tmp.append(x[i])
                i += 1
            out.append(np.average(tmp))
        return np.array(out)




In [2]:
import opc
import MDAnalysis as mda

u = mda.Universe('../dcd_big/min2.gro', '../dcd_big/skip_more.xtc')
halfz = u.dimensions[2] / 2


# Change selection to 'around' or 'not around' protein depending if you want far or close to protein
selection = ('resname POPC and not around 70 protein and prop z < %f' % halfz)
# selection = ('resname POPC and (around 70 protein) and not (around 40 protein) and prop z < %f' % halfz)
# selection = ('resname POPC  and prop z > %f' % halfz)

# selection_DOPE = ('resname DOPE and not around 10 protein and prop z < %f' % halfz)

OP_POPC1 = OrderParameters(u, opc.POPC1, 'POPC', selection)
POPC1 = OP_POPC1.compute_OP()

# OP_POPC2 = OrderParameters(u, opc.POPC2, 'POPC', selection)
# POPC2 = OP_POPC2.compute_OP()

# OP_DOPE1 = OrderParameters(u, opc.DOPE1, 'DOPE', selection_DOPE)
# DOPE1 = OP_DOPE1.compute_OP()

np.savetxt('POPC1_bottom_big_90nm.dat', POPC1)
# np.savetxt('POPC2_top_long.dat', POPC2)
# np.savetxt('DOPE1_bottom_1nm.dat', DOPE1)